In [ ]:
!pip install transformers==4.30 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

!pip install accelerate
!pip install bitsandbytes

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Learning Notebooks/

### Data Prep

In [ ]:
def read_text_file(file_path):
    try:
        with open(file_path, 'r',encoding='utf-8') as file:
            text = file.read()
        return text
    except FileNotFoundError:
        print("File not found!")
        return None
s2 = ''
for i in range(1977, 2001):
  file_content = read_text_file(f1+str(i)+".txt")
  s2+='. '+str(i)+ " year's letter below"
  s2+=file_content
s2

import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install pypdf
from pypdf import PdfReader

def ret_text(s, n, fix):
  reader = PdfReader('BH/'+str(i)+fix+'.pdf')
  text = ''
  for j in range(len(reader.pages)):
    page = reader.pages[j]
    text += page.extract_text()
  t = '.  '+str(i)+" year's letter below"
  return t+' '+text
  # return str(i)
s=''
for i in [2001, 2002]:
  s+=ret_text(s, i, 'pdf')

for i in range(2003, 2024):
  s+=ret_text(s, i, 'ltr')
s

def save_string_to_txt(string_data, file_path):
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(string_data)
        print("String saved to", file_path)
    except Exception as e:
        print("Error:", e)

save_string_to_txt(s2+s, f2)

String saved to BH_output_file.txt


### RAG

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
%%time
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from langchain.llms import HuggingFacePipeline

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

import accelerate
import bitsandbytes


model_id = "meta-llama/Llama-2-13b-chat-hf"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
        # max_length=200
        max_new_tokens = 1000
        )

def test_model(tokenizer, pipeline, prompt_to_test):

    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        # max_length=200,
        max_new_tokens = 1000
        )

    for seq in sequences:
        print('Result: ')
        nthelement = 20
        items = seq['generated_text'].split(' ')
        groups = []

        while items:
            first_three, items = items[:nthelement], items[nthelement:]
            groups.append(first_three)

        result = "\n".join(" ".join(g) for g in groups)
        print(result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

CPU times: user 53.5 s, sys: 1min 5s, total: 1min 59s
Wall time: 6min 39s


In [4]:
%%time
# testing
test_model(tokenizer,
           query_pipeline,
           "summarize what is the best way to measure any single year’s performance for any company")

Result: 
summarize what is the best way to measure any single year’s performance for any company?
Answer: I like to use a
combination of metrics to evaluate a company’s performance in a single year. These can include:
• Net income: This measure the
company’s overall profitability for the year, taking into account all revenue and expenses.
• Revenue growth: This measures the increase in
revenue from the previous year and shows whether the company is growing and expanding its business.
• Gross margin: This measures
the profitability of the company’s core operations, calculated as the difference between revenue and the cost of goods sold, expressed
as a percentage.
• Operating margin: This measures the profitability of the company’s operations, calculated as the difference between gross profit
and operating expenses, expressed as a percentage.
• Return on equity (ROE): This measures the company’s ability to generate profits from
shareholders’ equity, calculated as net income divided by sh

In [5]:
%%time

llm = HuggingFacePipeline(pipeline=query_pipeline)

llm(prompt="summarize what is the best way to measure any single year’s performance for any company")

CPU times: user 1min 15s, sys: 0 ns, total: 1min 15s
Wall time: 1min 18s


"?\n\nAnswer: The best way to measure a single year's performance for any company is to use a combination of financial and non-financial metrics that provide a comprehensive view of the company's performance. Here are some key metrics to consider:\n\n1. Revenue growth: Measure the year-over-year growth in revenue to assess the company's top-line performance.\n2. Net income growth: Measure the year-over-year growth in net income to assess the company's profitability.\n3. Earnings per share (EPS): Measure the year-over-year growth in EPS to assess the company's profitability from a shareholder's perspective.\n4. Return on equity (ROE): Measure the company's ROE to assess its ability to generate profits from shareholders' equity.\n5. Return on assets (ROA): Measure the company's ROA to assess its ability to generate profits from its assets.\n6. Gross margin: Measure the company's gross margin to assess its ability to generate profits from its sales.\n7. Operating margin: Measure the compa

In [12]:
%%time

llm = HuggingFacePipeline(pipeline=query_pipeline)

llm(prompt="List out some instances where warren buffett emphasizes on patience as a vertue of an investor")

CPU times: user 1min 7s, sys: 141 ms, total: 1min 7s
Wall time: 1min 13s


'.\n\nWarren Buffett, one of the most successful investors in history, has consistently emphasized the importance of patience as a key virtue for investors. Here are some instances where he has stressed the importance of patience:\n\n1. In his 1994 letter to shareholders, Buffett wrote, "Our favorite holding period is forever." This quote highlights the importance of patience in investing, as Buffett is willing to hold onto his investments for the long term, rather than trying to time the market or make quick profits.\n2. In a 2018 interview with CNBC, Buffett said, "I think the most important thing is to have a long-term perspective. The more you think about the short term, the more you\'re going to get hurt." This quote emphasizes the importance of patience in avoiding short-term thinking and focusing on the long-term potential of investments.\n3. In his 2017 letter to shareholders, Buffett wrote, "The weeds wither away, but the flowers remain." This quote is a metaphor for the impor

In [6]:
loader = TextLoader("BH_output_file.txt",
                    encoding="utf8")
documents = loader.load()

In [7]:
%%time
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")


retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

def test_rag(qa, query):
    print(f"Query: {query}\n")
    result = qa.run(query)
    print("\nResult: ", result)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

CPU times: user 1min 35s, sys: 1.62 s, total: 1min 37s
Wall time: 1min 48s


In [8]:
%%time

query = "summarize what is the best way to measure any single year’s performance for any company"
test_rag(qa, query)

Query: summarize what is the best way to measure any single year’s performance for any company



> Entering new RetrievalQA chain...

> Finished chain.

Result:  

The best way to measure any single year's performance for any company is to use the change in the company's per-share intrinsic value over that year, rather than the change in its stock price. This approach takes into account both realized and unrealized capital gains or losses, as well as extraordinary items, and presents a more accurate picture of the company's long-term economic performance. It is also more appropriate to use financial statements presenting equity securities at market value, as this provides a more comprehensive view of the company's performance. Additionally, it is important to keep in mind that the impact of various businesses on both absolute and relative performance can change over time, and to consider the many businesses that Berkshire Hathaway operates whose annual earnings are unaffected by chang

In [9]:
%%time
# doc sources

docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Query: summarize what is the best way to measure any single year’s performance for any company
Retrieved documents: 4
Source:  BH_output_file.txt
Text:  good arguments for simply using the change in our stock price. Over an extended period of time, in fact, that isthe best test. But year-to-year market prices can be extraordinarily erratic. Even evaluations covering as long as adecade can be greatly distorted by foolishly high or low prices at the beginning or end of the measurementperiod. Steve Ballmer, of Microsoft, and Jeff Immelt, of GE, can tell you about that problem, suffering as they dofrom the nosebleed prices at which their stocks traded when they were handed the managerial baton.
The ideal standard for measuring our yearly progress would be the change in Berkshire’s per-share intrinsic 

Source:  BH_output_file.txt
Text:  Long Term Results

     In measuring long term economic performance - in contrast to 
yearly performance - we believe it is appropriate to recognize 
fully

In [10]:
%%time
query = "List out some life lessons from berkshire hathaway annual letters that talk about human character values like consistency persuasion agility and lessons learnt \
from years of investing"
test_rag(qa, query)

Query: List out some life lessons from berkshire hathaway annual letters that talk about human character values like consistency persuasion agility and lessons learnt from years of investing



> Entering new RetrievalQA chain...

> Finished chain.

Result:  

1. Consistency: Warren Buffett emphasizes the importance of consistency in his letters, highlighting the need for Berkshire Hathaway's managers to maintain a consistent approach to their businesses, even in the face of challenges and setbacks. This lesson can be applied to life by focusing on the long-term and being consistent in one's actions and decisions.

2. Persuasion: Buffett also stresses the importance of persuasion in his letters, noting that successful managers must be able to persuade others to follow their vision and goals. This lesson can be applied to life by learning how to effectively communicate one's ideas and values to others.

3. Agility: Buffett highlights the need for agility in Berkshire Hathaway's business

In [11]:
%%time
query = "List out some instances where warren buffett emphasizes on patience as a vertue of an investor"
test_rag(qa, query)

Query: List out some instances where warren buffett emphasizes on patience as a vertue of an investor



> Entering new RetrievalQA chain...

> Finished chain.

Result:  

Warren Buffett emphasizes patience as a virtue of an investor in the following instances:

1. He highlights the importance of patience in his 1980 letter to shareholders, stating that "an institution will resist any change in its current direction" and that "corporate projects or acquisitions will materialize to soak up available funds."
2. He notes that "any business craving of the leader, however foolish, will be quickly supported by detailed rate-of-return and strategic studies prepared by his troops" and that "the behavior of peer companies, whether they are expanding, acquiring, setting executive compensation or whatever, will be mindlessly imitated."
3. He stresses the importance of patience in his decision-making, stating that "I have tried to organize and manage Berkshire in ways that minimize its influence" 